# Cosmos DB in Fabric

## Vector Search Sample

This sample notebook shows how to do vector search using Cosmos DB in Fabric with Fabric's built-in OpenAI text embedding model. The sample dataset is a product catalog is contains a `vectors` property containing the vectorized name, category and description properties.

### Features of this Notebook
This notebook demonstrates the following concepts:

- How to write a query in Cosmos DB in Fabric to perform a vector similarity search
- How to use the similarity score from a Cosmos DB vector search to filter and sort for the best results
- How to generate embeddings using Fabric's built-in OpenAI model, text-embedding-ada-002

Requirements:
- This sample utilizes the SampleVectorData dataset in Cosmos DB in Fabric Data Explorer. Create a new Cosmos DB artifact, then on the Home screen after creating a new artifact, click SampleVectorData to create the new SampleVectorData container.


In [ ]:
#Install packages
%pip install azure-cosmos
%pip install openai

In [ ]:
#Imports and config values
import logging
from rich.pretty import pprint
from typing import List, Dict, Any, Optional

import openai
from azure.cosmos.aio import CosmosClient
from azure.cosmos.exceptions import CosmosHttpResponseError

os.environ["OPENAI_API_VERSION"] = "2023-05-15"
OPEN_AI_MODEL = "text-embedding-ada-002"

COSMOS_ENDPOINT = '{your-cosmos-endpoint}'
COSMOS_DATABASE_NAME = 'cosmos-sample-database'
COSMOS_CONTAINER_NAME = 'SampleVectorData'

In [ ]:
# Custom TokenCredential implementation for Fabric authentication
%pip install azure-core
from azure.core.credentials import TokenCredential, AccessToken
import base64
import json
import notebookutils
from datetime import datetime, timezone

class FabricTokenCredential(TokenCredential):

    def get_token(self, *scopes: str, claims: Optional[str] = None, tenant_id: Optional[str] = None,
                  enable_cae: bool = False, **kwargs: Any) -> AccessToken:
        access_token = notebookutils.credentials.getToken("https://cosmos.azure.com/")
        parts = access_token.split(".")
        if len(parts) < 2:
            raise ValueError("Invalid JWT format")
        payload_b64 = parts[1]
        # Fix padding
        padding = (-len(payload_b64)) % 4
        if padding:
            payload_b64 += "=" * padding
        payload_json = base64.urlsafe_b64decode(payload_b64.encode("utf-8")).decode("utf-8")
        payload = json.loads(payload_json)
        exp = payload.get("exp")
        if exp is None:
            raise ValueError("exp claim missing in token")
        return AccessToken(token=access_token, expires_on=exp) 

In [ ]:
# Initialize Cosmos DB client and container
COSMOS_CLIENT = CosmosClient(COSMOS_ENDPOINT, FabricTokenCredential())
DATABASE = COSMOS_CLIENT.get_database_client(COSMOS_DATABASE_NAME)
CONTAINER = DATABASE.get_container_client(COSMOS_CONTAINER_NAME)

In [ ]:
# Define function to generate embeddings for vector search
def generate_embeddings(text):

    response = openai.embeddings.create(
        input=text,
        model=OPEN_AI_MODEL
    )
    
    embeddings = response.model_dump()
    return embeddings['data'][0]['embedding']

# Test Fabric's OpenAI model
#search_text = "Hello from Fabric Notebooks"
#embeddings = generate_embeddings(search_text.strip())
#print(embeddings)

In [ ]:
#Define function to perform vector search
async def search_products(search_text: str, similarity: float = 0.8, limit: int = 5) -> List[Dict[str, Any]]:

    try:
        # Generate embeddings for the search text
        embeddings = generate_embeddings(search_text.strip())

        # Cosmos query with VectorDistance to perform similarity search
        query = """
            SELECT TOP @limit 
                VectorDistance(c.vectors, @embeddings) AS SimilarityScore,
                c.name, 
                c.description,
                c.categoryName,
                c.currentPrice,
                c.inventory,
                c.priceHistory
            FROM c 
            WHERE 
                c.docType = @docType AND
                VectorDistance(c.vectors, @embeddings) >= @similarity
            ORDER BY 
                VectorDistance(c.vectors, @embeddings)
        """

        parameters = [
            {"name": "@limit", "value": limit},
            {"name": "@embeddings", "value": embeddings},
            {"name": "@docType", "value": "product"},
            {"name": "@similarity", "value": similarity}
        ]

        # Async query: gather results into a list
        products = [p async for p in CONTAINER.query_items(
            query=query,
            parameters=parameters
        )]

        # Always remove any properties with vectors in them, they are unnecessary and extremely large
        for p in products:
            p.pop('vectors', None)

        return products
        
    except CosmosHttpResponseError as e:
        logging.error(f"Cosmos DB query failed: {e}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error in search_products: {e}")
        raise

In [ ]:
# Vector Search for products
# The value for similarity score returns 6 products, the limit will restrict the results to 5
# Feel free to adjust these to see how the results change. 
# You can also modify the search text for different results
products = await search_products(search_text="gaming pc", similarity=0.80, limit=5)

#print the number of products found
print(f"Found {len(products)} products matching the search criteria.")

display(products) # for tabular output
pprint(products) #Json friendly output